In [1]:
import pandas
import pickle

merged_data = pandas.read_pickle('../data/data-preperation-output.pickle')
merged_data.head()

In [ ]:
import math

# analyze the max weight of a course
assessment_lists_per_student_per_course = merged_data.groupby(['code_module', 'code_presentation']).agg({
    "assessments": lambda x: list(x),
}).reset_index()

def analyze_max_weight_per_course_per_presentation(row):
    asessments_of_students = row['assessments']
    max_weight = 0
    for asessments_of_student in asessments_of_students:
        weights = [weight for (id_assessment, date_submitted, score, assessment_type, date, weight) in asessments_of_student]
        student_max_weight = sum(weights)
        if student_max_weight > max_weight:
            max_weight = student_max_weight
        

    row['max_weight'] = max_weight
    return row

max_weights_per_course_per_presentation = assessment_lists_per_student_per_course.apply(analyze_max_weight_per_course_per_presentation, axis = 1)
max_weights_per_course_per_presentation = max_weights_per_course_per_presentation.drop('assessments', axis=1).set_index(['code_module', 'code_presentation'])
max_weights_per_course_per_presentation.head(21)

In [ ]:
def analyze_average_grade(assessments):
    total_weight = 0
    total_score = 0

    for assessment in assessments:
        id_assessment, date_submitted, score, assessment_type, date, weight = assessment
        if (not(type(score) == float)):
            print(score)
        total_weight +=  weight
        total_score += score * weight


    if total_weight > 0:
        average_score = total_score / total_weight
        return average_score
    else:
        return math.nan

merged_data['average_score'] = merged_data['assessments'].transform(analyze_average_grade)
merged_data.head()

In [ ]:
# jort
# sum of all clicks

def sum_clicks_vles(vles):
    sum_clicks = 0
    for [id_site, date, activity_type, sum_click] in vles:
        sum_clicks += sum_click
    return sum_clicks

merged_data["sum_clicks"] = merged_data["vles"].transform(sum_clicks_vles)
merged_data.head()

In [ ]:
merged_data = merged_data.drop(["assessments", "vles"], axis=1)
merged_data.head()

In [ ]:
merged_data.to_pickle("../data/data-extension-output.pickle")